In [33]:
from transformers import pipeline, set_seed
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric, concatenate_datasets

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup

from tqdm import tqdm
import torch
import numpy as np
import random

from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType

In [2]:
import os
device = 'cuda' 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
def set_seed(args):
    """
    :param args:
    :return:
    """
    torch.manual_seed(args)
    torch.cuda.manual_seed_all(args)
    np.random.seed(args)
    random.seed(args)
    torch.backends.cudnn.deterministic = True

random_seed = 42
set_seed(random_seed)

In [4]:
# 加载数据集
dataset = load_dataset('samsum')

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Found cached dataset samsum (/home/is/kaifan-l/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 14732
Test dataset size: 819


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [6]:
split_lengths = [len(dataset[split]) for split in dataset]
split_lengths

[14732, 819, 818]

In [7]:
# 加载tokenizer

checkpoint = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [8]:
# 定义模型
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, 
                                 inference_mode=False,
                                 num_virtual_tokens=20,) # 所插入的虚拟token数量
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model = get_peft_model(model, peft_config).to(device)
model.print_trainable_parameters() 

trainable params: 983,040 || all params: 784,133,120 || trainable%: 0.1253664683874085


In [9]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i: i+batch_size]

In [13]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article",
                                column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                           padding="max_length", return_tensors="pt")

        # model.generate是huggingface提供的一种文本生成的方法之一
        # 用于基于给定的输入生成相应的输出
        # length_penalty: 长度惩罚
        # max_length: 生成文本的最大长度
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                             clean_up_tokenization_spaces=True)
                             for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [11]:
device

'cuda'

In [ ]:
rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(dataset['test'], rouge_metric, model, tokenizer,
                                    column_text = 'dialogue', column_summary='summary', batch_size=8)
score

In [15]:
dataset['train'][:10]

{'id': ['13818513',
  '13728867',
  '13681000',
  '13730747',
  '13728094',
  '13716343',
  '13611672',
  '13730463',
  '13809976',
  '13809912'],
 'dialogue': ["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
  'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
  "Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style",
  "Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: 

In [16]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

dataset_pt = dataset.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [17]:
dataset_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [18]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
model.print_trainable_parameters()

trainable params: 983,040 || all params: 784,133,120 || trainable%: 0.1253664683874085


In [21]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='saved',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt['train'],
                  eval_dataset=dataset_pt['validation'])

In [ ]:
trainer.train()

In [23]:
model.save_pretrained('t5-samsum/model')
tokenizer.save_pretrained('t5-samsum/tokenizer')

('t5-samsum/tokenizer/tokenizer_config.json',
 't5-samsum/tokenizer/special_tokens_map.json',
 't5-samsum/tokenizer/spiece.model',
 't5-samsum/tokenizer/added_tokens.json',
 't5-samsum/tokenizer/tokenizer.json')